In [1]:
import os
from datetime import datetime as dt
from random import randint
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')

import torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from tensorboardX import SummaryWriter
from torchsummaryX import summary

from hparams import Hparam
from data.dataset import SpeechDataset
from CPC_true_NCE.model import CPCModel_NCE
# from CPC_classifiers.speaker_model import SpeakerClassificationCPC

In [2]:
config = Hparam('./CPC_true_NCE/config.yaml')
# config.train.device = 'cpu'
gettime = lambda: str(dt.time(dt.now()))[:8]

## model

In [3]:
ds = SpeechDataset(config.data.path)
train_ds, test_ds = train_test_split(ds, test_size=0.2)

100%|██████████| 2703/2703 [00:04<00:00, 566.00it/s]


In [4]:
model = CPCModel_NCE(config).to(config.train.device)

In [5]:
sample = ds[27][1].unsqueeze(0).unsqueeze(0).expand(4, 1, 20480).to(config.train.device)
sample.size()

torch.Size([4, 1, 20480])

In [6]:
with torch.no_grad():
    logits, labels = model(sample)

print('logits', [t.size() for t in logits])

fk torch.Size([2032, 1]) ans torch.Size([2032, 1])
fk torch.Size([2016, 1]) ans torch.Size([2016, 1])
fk torch.Size([2000, 1]) ans torch.Size([2000, 1])
fk torch.Size([1984, 1]) ans torch.Size([1984, 1])
fk torch.Size([1968, 1]) ans torch.Size([1968, 1])
fk torch.Size([1952, 1]) ans torch.Size([1952, 1])
fk torch.Size([1936, 1]) ans torch.Size([1936, 1])
fk torch.Size([1920, 1]) ans torch.Size([1920, 1])
fk torch.Size([1904, 1]) ans torch.Size([1904, 1])
fk torch.Size([1888, 1]) ans torch.Size([1888, 1])
fk torch.Size([1872, 1]) ans torch.Size([1872, 1])
fk torch.Size([1856, 1]) ans torch.Size([1856, 1])
logits [torch.Size([2032, 1]), torch.Size([2016, 1]), torch.Size([2000, 1]), torch.Size([1984, 1]), torch.Size([1968, 1]), torch.Size([1952, 1]), torch.Size([1936, 1]), torch.Size([1920, 1]), torch.Size([1904, 1]), torch.Size([1888, 1]), torch.Size([1872, 1]), torch.Size([1856, 1])]


In [7]:
logs = logits[0]
labs = labels[0]

In [8]:
logs.size(), labs.size()

(torch.Size([2032, 1]), torch.Size([2032, 1]))

In [11]:
logs.size()

torch.Size([2032, 1])

In [15]:


roc_auc_score(to_numpy(labs), to_numpy(logs))

0.5178043689420713

In [32]:
labls = (to_numpy(logs) > 0.5).astype(int)
accuracy_score(to_numpy(labs), labls)

0.6299212598425197